# Imports

In [ ]:
import torch
import pandas as pd
import numpy as np
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Classes

In [ ]:
class LSTMReservoir(nn.Module):
    def __init__(self, input_size, reservoir_size, output_size, num_layers=1, pred_len=1):
        super(LSTMReservoir, self).__init__()
        self.input_size = input_size
        self.reservoir_size = reservoir_size
        self.output_size = output_size
        self.pred_len = pred_len
        self.num_layers = num_layers

        # LSTM as reservoir
        self.lstm = nn.LSTM(input_size, reservoir_size, num_layers=num_layers, batch_first=True)
        # freeze reservoir
        for param in self.lstm.parameters():
            param.requires_grad = False

        # Output weights
        self.linear1 = nn.Linear(reservoir_size, 64)
        self.linear2 = nn.Linear(64, output_size)

        self.dropout = nn.Dropout(0.2)

    # LSTM forward pass
    def forward(self, x):

        # input shape: (amount of sequences, sequences length, dimensionality of problem)
        input_len = x.size(1)
        for i in range(self.pred_len):
            # get the input and the previous outputs
            input = x[:, i:i+input_len, :]

            # the output will be just on the last hidden state
            h, _ = self.lstm(input)
            h = h[:, -1, :]
            out = F.leaky_relu(self.linear1(h))
            out = self.dropout(out)
            out = self.linear2(out)

            out = out.unsqueeze(1)
            x = torch.cat((x, out), dim=1)

        x = x[:, -self.pred_len:, :]
        return x

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, reservoir_size, output_size, num_layers=1, pred_len=1):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.reservoir_size = reservoir_size
        self.output_size = output_size
        self.pred_len = pred_len
        self.num_layers = num_layers

        # LSTM as reservoir
        self.lstm = nn.LSTM(input_size, reservoir_size, num_layers=num_layers, batch_first=True)

        # Output weights
        self.linear1 = nn.Linear(reservoir_size, 64)
        self.linear2 = nn.Linear(64, output_size)

        self.dropout = nn.Dropout(0.2)

    # LSTM forward pass
    def forward(self, x):

        # input shape: (amount of sequences, sequences length, dimensionality of problem)
        input_len = x.size(1)
        for i in range(self.pred_len):
            # get the input and the previous outputs
            input = x[:, i:i+input_len, :]

            # the output will be just on the last hidden state
            h, _ = self.lstm(input)
            h = h[:, -1, :]
            out = F.leaky_relu(self.linear1(h))
            out = self.dropout(out)
            out = self.linear2(out)

            out = out.unsqueeze(1)
            x = torch.cat((x, out), dim=1)

        x = x[:, -self.pred_len:, :]
        return x

In [ ]:
def evaluate(num_epochs, criterion, optimizer, currentModel, train_dataloader, val_dataloader, device, scheduler=None):
    train_losses = []
    val_best_loss = np.inf
    val_best_results = {'inputs':[], 'predictions':[], 'targets':[], 'losses':[]}

    for epoch in range(num_epochs):

        running_loss = []
        
        val_results = {'inputs':[], 'predictions':[], 'targets':[], 'losses':[]}

        # Train the model
        currentModel.train()
        for inputs, targets in train_dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = currentModel(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss.append(loss.item())

        # Evaluate the model
        currentModel.eval()
        with torch.no_grad():
            for val_inputs, val_targets in val_dataloader:
                val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)
                val_predictions = currentModel(val_inputs)
                val_loss = criterion(val_predictions, val_targets)
                val_results['inputs'].append(val_inputs)
                val_results['predictions'].append(val_predictions)
                val_results['targets'].append(val_targets)
                val_results['losses'].append(val_loss.cpu().item())

            val_mean_loss = np.mean(val_results["losses"])
            if val_mean_loss < val_best_loss:
                val_best_loss = val_mean_loss
                val_best_results = val_results
                print("!!! BEST MODEL !!!")

        # end of epoch  
        train_losses.append(np.mean(running_loss))
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {np.mean(running_loss):.3f}, Validation loss: {val_mean_loss:.3f}')
        if scheduler is not None:
            print("Learning rate: %.5f" % scheduler.get_last_lr()[0])
            scheduler.step() 

    return val_best_results, train_losses


# Code

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Working on:", device)
print(30*"-")

# Define sequences length
pred_len = 100
input_len = 400

# Define the model parameters
io_size = 2
reservoir_size = 8
num_epochs = 20

dimensionality = 2

### LOAD DATA
# Load the data
print("Loading data...")
train_t, train_dataloader, val_t, val_dataloader = loadData(dimensionality, pred_len, input_len)
print("Train batches:", len(train_dataloader))
print("Train input sequences:", len(train_dataloader.dataset))
print("Validation batches:", len(val_dataloader))
print("Validation input sequences:", len(val_dataloader.dataset))
print(30*"-")

In [ ]:
# init the models
model = LSTMReservoir(io_size, reservoir_size, io_size, num_layers=1, pred_len=pred_len).to(device)
modelBenchmark = LSTM(io_size, reservoir_size, io_size, num_layers=1, pred_len=pred_len).to(device)

# NMSE weighted as criterion
def NormalizedMeanSquaredError(y_pred, y_true):
    device = y_pred.get_device()
    if device == -1:
        device = 'cpu'
    pred_len = y_pred.size(1)
    batch_size = y_pred.size(0)

    squared_dist = torch.sum((y_true - y_pred)** 2, dim=2) # squared euclidean distances between predictions
    true_squared_norm = torch.sum(y_true ** 2, dim=2)
    nmse = squared_dist / true_squared_norm
    # actual (from above) shape: (batch size, prediction length)
    # as a neutral transformation for an overall error just take the mean on the prediction length and then on the batch size
    # WEIGHTED
    weights = torch.arange(start=1,end=pred_len+1,step=1).flip(dims=(0,)).square().to(device)
    weights = weights/weights.sum()
    aggregated_nmse = torch.zeros(batch_size)
    for batch in range(batch_size):
        aggregated_nmse[batch] = torch.dot(nmse[batch], weights)
    # aggregated_nmse = torch.mean(torch.mean(nmse, dim=1), dim=0) # UNWEIGHTED
    aggregated_nmse = torch.mean(aggregated_nmse, dim=0)
    return aggregated_nmse

# Training

In [ ]:
### RESERVOIR
# Define training setup
# criterion
criterion = NormalizedMeanSquaredError
# optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)
# scheduler
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
print("Reservoir training...")
# start counting the time
start = time.time()
# Train the model
val_results, train_losses = (
    evaluate(num_epochs, criterion, optimizer, model, train_dataloader, val_dataloader, device, scheduler))
# stop counting the time
end = time.time()
print('Time elapsed: ', end - start, "s")
print(30*"-")

# plot training loss
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training loss')
plt.savefig('Media/3BP_training_loss.png')
plt.close()
    

### BENCHMARK MODEL
print("Benchmark training...")
# training setup
# criterion
criterion = NormalizedMeanSquaredError
# optimizer
optimizer = optim.Adam(modelBenchmark.parameters(), lr=0.001)
# scheduler
scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
# start counting the time
start = time.time()
# Train the benchmark model
val_results_benchmark, train_losses_benchmark = (
    evaluate(num_epochs, criterion, optimizer, modelBenchmark, train_dataloader, val_dataloader, device, scheduler))
# stop counting the time
end = time.time()
print('Time elapsed: ', end - start, "s")
print(30*"-")

# plot training loss
plt.plot(train_losses_benchmark)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training loss')
plt.savefig('Media/3BP_training_loss_benchmark.png')
plt.close()

# Plots

In [ ]:
# Plotting the predictions
plt.figure(figsize=(15, 15))

how_many_plots = min(4, len(val_dataloader.dataset))
n_sequences = len(val_dataloader.dataset)
sequence_to_plot = torch.randint(0, n_sequences, (how_many_plots,))

batch_size = val_results['targets'][0].size(0)
batch_to_plot = torch.randint(0, batch_size, (how_many_plots,))

for plot in range(how_many_plots - how_many_plots%2):
    seq = sequence_to_plot[plot].item()
    batch = batch_to_plot[plot].item()
    # Plotting the predictions
    plt.subplot(how_many_plots // 2, 2, plot + 1)
    plt.plot(val_results['inputs'][seq][batch,:,0].cpu(), val_results['inputs'][seq][batch,:,1].cpu(), label='Input')
    plt.plot(val_results['targets'][seq][batch,:,0].cpu(), val_results['targets'][seq][batch,:,1].cpu(), label='Target')
    plt.plot(val_results['predictions'][seq][batch,:,0].cpu(), val_results['predictions'][seq][batch,:,1].cpu(), label='Predicted (Reservoir)')
    plt.plot(val_results_benchmark['predictions'][seq][batch,:,0].cpu(), val_results_benchmark['predictions'][seq][batch,:,1].cpu(), label='Predicted (Benchmark)')
    plt.xlabel('Time step')
    plt.legend()
    plt.grid()

plt.tight_layout()
if diego:
    plt.savefig('D:/File_vari/Scuola/Universita/Bicocca/Magistrale/AI4ST/23-24/II_semester/AIModels/3_Body_Problem/RestrictedThreeBodyProblem/Media/3BP_prediction.png')
else:
    plt.savefig('Media/3BP_prediction.png')
plt.close()